In [1]:
import numpy as np
from scipy import stats

In [11]:
n = 201
x = np.linspace(0, 20, n)
w = np.array([-6, 0.75])
y = w[0] + w[1] * x + stats.norm.rvs(size = n)

def regress(n, x, y, sx, sy, sxx, sxy):
    sx += x
    sy += y
    sxx += x * x
    sxy += x * y
    sx2 = sx * sx
    if n == 0 or sxx == sx2:
        w1 = w0 = 0
    else:
        w1 = (sxy - sx * sy / n) / (sxx - sx2 / n)
        w0 = (sy - w1 * sx) / n
    return w0, w1, sx, sy, sxx, sxy

sx = sy = sxx = sxy = 0.0

for i in range(n):
    w0, w1, sx, sy, sxx, sxy = regress(i, x[i], y[i], sx, sy, sxx, sxy)

print w1

xbar = np.mean(x)
ybar = np.mean(y)
print (sum(x * y) - n * xbar * ybar) / (sum(x ** 2) - n * xbar ** 2)

0.776289734767
0.766974770607
